In [1]:
import os
import time
import copy
import argparse
import numpy as np
import torch
from fvcore.nn import FlopCountAnalysis
import torch.nn as nn
from torchvision.utils import save_image
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
def train(model, train_loader, test_loader, optimizer, scheduler, criterion, epochs):
    model = model.to(device)
    criterion = criterion.to(device)
    model.train()
    
    for epoch in range(epochs):
        loss_avg, acc_avg, num_exp = 0, 0, 0
        for i_batch, datum in enumerate(train_loader):
            img = datum[0].float().to(device)
            lab = datum[1].long().to(device)
            n_b = lab.shape[0]
            output = model(img)
            loss = criterion(output, lab)
            acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))
            loss_avg += loss.item()*n_b
            acc_avg += acc
            num_exp += n_b

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            del img, lab
                
        loss_avg /= num_exp
        acc_avg /= num_exp
        scheduler.step()
        
    loss_avg, acc_avg, num_exp = 0, 0, 0   
    model.eval()
    for i_batch, datum in enumerate(test_loader):
        img = datum[0].float().to(device)
        lab = datum[1].long().to(device)
        n_b = lab.shape[0]
        output = model(img)
        loss = criterion(output, lab)
        acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))
        loss_avg += loss.item()*n_b
        acc_avg += acc
        num_exp += n_b
        
        del img, lab

    loss_avg /= num_exp
    acc_avg /= num_exp
      
    return loss_avg, acc_avg

MNIST_dataset = 'MNIST'
MNIST_data_path = './MNISTdata'
MNIST_channel, MNIST_im_size, MNIST_num_classes, MNIST_class_names, MNIST_mean, MNIST_std, MNIST_dst_train, MNIST_dst_test, MNIST_testloader = get_dataset(MNIST_dataset, MNIST_data_path)
MNIST_trainloader = torch.utils.data.DataLoader(MNIST_dst_train, batch_size=8, shuffle=True, num_workers=0)
criterion = nn.CrossEntropyLoss()
epochs = 20
model_set = ['LeNet', 'AlexNet', 'VGG11', 'ConvNetD4']
iteration_set = ['first', 'second', 'third']

for model_architecture in model_set:
    print('========================================')
    accs_each_model = []
    training_times_each_model = []
    for itr in iteration_set:
        print('Start training on the '+itr+' '+model_architecture+' architecture on the original MNIST dataset')
        model = get_network(model_architecture, MNIST_channel, MNIST_num_classes, MNIST_im_size).to(device) # get a random model
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0)
        start = time.time()
        _, acc_avg = train(model, MNIST_trainloader , MNIST_testloader , optimizer, scheduler, criterion, epochs)
        elapsed_time = time.time()-start
        training_times_each_model.append(elapsed_time)
        accs_each_model.append(acc_avg*100)
        print('Training takes {} seconds'.format(training_times_each_model[-1]))
        print('Test accuracy is {}%'.format(accs_each_model[-1]))
    print('----------------------------------------')
    print('Average training time is {} seconds'.format(sum(training_times_each_model)/len(training_times_each_model)))
    print('Average test accuracy is {}%'.format(sum(accs_each_model)/len(accs_each_model)))


Start training on the first LeNet architecture on the original MNIST dataset
Training takes 670.8751773834229 seconds
Test accuracy is 99.24%
Start training on the second LeNet architecture on the original MNIST dataset
Training takes 650.0438580513 seconds
Test accuracy is 99.22999999999999%
Start training on the third LeNet architecture on the original MNIST dataset
Training takes 647.1453673839569 seconds
Test accuracy is 99.29%
----------------------------------------
Average training time is 656.0214676062266 seconds
Average test accuracy is 99.25333333333333%
Start training on the first AlexNet architecture on the original MNIST dataset
Training takes 855.1789402961731 seconds
Test accuracy is 99.42%
Start training on the second AlexNet architecture on the original MNIST dataset
Training takes 865.210129737854 seconds
Test accuracy is 99.44%
Start training on the third AlexNet architecture on the original MNIST dataset
Training takes 869.5471694469452 seconds
Test accuracy is 99.